In [2]:
import torch
import numpy as np
import os
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

In [3]:
device="mps" if torch.backends.mps.is_available() else "cpu"
device

'mps'

In [4]:
n="80"
DATA_PATH=f"data/Embedings/Embeding-{n}"
TRAIN_EMBEDS = os.path.join(DATA_PATH, f'trainEmbeds{n}.npz')
TEST_EMBEDS = os.path.join(DATA_PATH, f'testEmbeds{n}.npz')
trainEmbeds, trainLabels = np.load(TRAIN_EMBEDS, allow_pickle=True).values()
testEmbeds, testLabels = np.load(TEST_EMBEDS, allow_pickle=True).values()
trainEmbeds.shape

(76626, 512)

In [5]:
import pickle
with open('data/class_list.pkl', 'rb') as file:
    ClassList = pickle.load(file)
print(len(ClassList))    
def name_from_index(i, ClassList=ClassList):
    reversed_dict = {v: k for k, v in ClassList.items()}
    if isinstance(i, np.ndarray):  # Check if i is a NumPy array
        names = [reversed_dict.get(idx.item(), None) for idx in i]
        return names
    else:
        return reversed_dict.get(i, None)

19


In [6]:
import torch.nn as nn
import torch.optim as optim
X = torch.tensor(trainEmbeds, dtype=torch.float32).to(device)
y = torch.tensor(trainLabels, dtype=torch.long).to(device)
X_test=torch.tensor(testEmbeds,dtype=torch.float32).to(device)
y_test=torch.tensor(testLabels,dtype=torch.long).to(device)

# Defining SVM model

In [7]:
class SoftmaxUsed(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(nn.Linear(512, 4096),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(4096, 1024),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(1024, len(ClassList)),
                                 nn.LogSoftmax(dim=1))
    def forward(self, x):
        return self.layers(x)

class SVM(nn.Module):
    def __init__(self):
        super(SVM, self).__init__()
        self.fc = nn.Linear(256, len(ClassList))
        self.layers = nn.Sequential(nn.Linear(512, 256),
                                    nn.ReLU(),
                                    nn.Dropout(0.6),)

    def forward(self, x):
        return self.fc(self.layers(x)).pow(3)
# class SVM(nn.Module):
#     def __init__(self):
#         super(SVM, self).__init__()
#         self.fc = nn.Sequential(nn.Linear(X.shape[1], len(ClassList)),
#                                 nn.Dropout(0.7))

#     def forward(self, x):
#         return self.fc(x)

# model = SoftmaxUsed().to(device)
model=SVM().to(device)
# model.load_state_dict(torch.load(f="data/models/predictFacesModelAcc91.70_With40Embeds.pth"))

# Loss function and Optimizer function

In [8]:
loss_func = nn.CrossEntropyLoss()
# nn.NLLLoss()
admoptimizer = optim.Adam(model.parameters(), lr=0.000051)

# Accuracy Function

In [9]:
def acc(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

# Training Loop

In [10]:
def modelSave(model):
    from pathlib import Path

    # 1. Create models directory 
    MODEL_PATH = Path(f"data/classLen- {len(ClassList)}/embeding-{n}")
    MODEL_PATH.mkdir(parents=True, exist_ok=True)

    # 2. Create model save path 
    MODEL_NAME = f"predictFacesModelAcc{testAcc:.2f}_With{n}Embeds.pth"
    MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

    # 3. Save the model state dict 
    print(f"Saving model to: {MODEL_SAVE_PATH}")
    torch.save(obj=model.state_dict(), # only saving the state_dict() only saves the models learned parameters
               f=MODEL_SAVE_PATH)

In [ ]:
%%time
num_epochs = 2400000
trainingLoss= []
testingLoss =1
trainAcc = []
testAcc =0
for epoch in range(num_epochs):
    outputs = model(X)
    _, y_predict = torch.max(outputs, 1)
    train_accuracy = acc(y, y_predict)
    train_loss = loss_func(outputs, y)

    admoptimizer.zero_grad()
    train_loss.backward()
    admoptimizer.step()
    model.eval()
    with torch.no_grad():
        predictions = model(X_test)
        _, pred_labels = torch.max(predictions, 1)
        y_test = y_test.to(torch.long)  # Convert to Long data type
        test_accuracy = acc(y_test, pred_labels)
        test_loss = loss_func(predictions, y_test)
    if epoch % 10== 0:
        if testAcc<test_accuracy:
            testAcc=test_accuracy
            if testAcc>96.5:
                modelSave(model)
        if test_loss<testingLoss and testAcc>96.5:
            testingLoss=test_loss-0.001
            modelSave(model)
            
        print(f"Epoch: {epoch} | TrainingLoss= {train_loss} | TestingLoss= {test_loss} | Accuracy= {test_accuracy}")


Epoch: 0 | TrainingLoss= 2.7919223308563232 | TestingLoss= 2.551236391067505 | Accuracy= 4.658385093167702
Epoch: 10 | TrainingLoss= 2.791900634765625 | TestingLoss= 2.5512030124664307 | Accuracy= 4.968944099378882
Epoch: 20 | TrainingLoss= 2.7918615341186523 | TestingLoss= 2.551142454147339 | Accuracy= 4.968944099378882
Epoch: 30 | TrainingLoss= 2.791790246963501 | TestingLoss= 2.5510241985321045 | Accuracy= 5.900621118012422
Epoch: 40 | TrainingLoss= 2.7916572093963623 | TestingLoss= 2.5507984161376953 | Accuracy= 6.832298136645963
Epoch: 50 | TrainingLoss= 2.791421413421631 | TestingLoss= 2.550394296646118 | Accuracy= 6.832298136645963
Epoch: 60 | TrainingLoss= 2.7910215854644775 | TestingLoss= 2.549710750579834 | Accuracy= 6.832298136645963
Epoch: 70 | TrainingLoss= 2.790363311767578 | TestingLoss= 2.548586130142212 | Accuracy= 7.453416149068323
Epoch: 80 | TrainingLoss= 2.7893097400665283 | TestingLoss= 2.5467751026153564 | Accuracy= 9.006211180124224
Epoch: 90 | TrainingLoss= 2.7

Epoch: 760 | TrainingLoss= 1.9753000736236572 | TestingLoss= 1.3042082786560059 | Accuracy= 48.75776397515528
Epoch: 770 | TrainingLoss= 1.9714441299438477 | TestingLoss= 1.3009215593338013 | Accuracy= 48.75776397515528
Epoch: 780 | TrainingLoss= 1.9676578044891357 | TestingLoss= 1.297658920288086 | Accuracy= 48.75776397515528
Epoch: 790 | TrainingLoss= 1.9639296531677246 | TestingLoss= 1.294406771659851 | Accuracy= 48.4472049689441
Epoch: 800 | TrainingLoss= 1.960236668586731 | TestingLoss= 1.2911403179168701 | Accuracy= 48.4472049689441
Epoch: 810 | TrainingLoss= 1.9565231800079346 | TestingLoss= 1.2877869606018066 | Accuracy= 48.4472049689441
Epoch: 820 | TrainingLoss= 1.9526302814483643 | TestingLoss= 1.2841415405273438 | Accuracy= 48.4472049689441
Epoch: 830 | TrainingLoss= 1.948093295097351 | TestingLoss= 1.2795050144195557 | Accuracy= 48.4472049689441
Epoch: 840 | TrainingLoss= 1.9417470693588257 | TestingLoss= 1.2719367742538452 | Accuracy= 48.4472049689441
Epoch: 850 | Trainin

Epoch: 1510 | TrainingLoss= 1.1746437549591064 | TestingLoss= 0.3283645212650299 | Accuracy= 77.01863354037268
Epoch: 1520 | TrainingLoss= 1.166839361190796 | TestingLoss= 0.3286409080028534 | Accuracy= 77.32919254658384
Epoch: 1530 | TrainingLoss= 1.1594082117080688 | TestingLoss= 0.32720986008644104 | Accuracy= 77.32919254658384
Epoch: 1540 | TrainingLoss= 1.1523113250732422 | TestingLoss= 0.325858473777771 | Accuracy= 77.32919254658384
Epoch: 1550 | TrainingLoss= 1.1454812288284302 | TestingLoss= 0.32512518763542175 | Accuracy= 77.32919254658384
Epoch: 1560 | TrainingLoss= 1.1388866901397705 | TestingLoss= 0.3244999945163727 | Accuracy= 77.32919254658384
Epoch: 1570 | TrainingLoss= 1.1324896812438965 | TestingLoss= 0.3236478865146637 | Accuracy= 77.32919254658384
Epoch: 1580 | TrainingLoss= 1.126262903213501 | TestingLoss= 0.32261258363723755 | Accuracy= 77.32919254658384
Epoch: 1590 | TrainingLoss= 1.12018883228302 | TestingLoss= 0.3215889036655426 | Accuracy= 77.32919254658384
Epo

Epoch: 2250 | TrainingLoss= 0.8354107737541199 | TestingLoss= 0.2017303854227066 | Accuracy= 79.8136645962733
Epoch: 2260 | TrainingLoss= 0.8320395350456238 | TestingLoss= 0.20055891573429108 | Accuracy= 79.8136645962733
Epoch: 2270 | TrainingLoss= 0.8287009596824646 | TestingLoss= 0.19928056001663208 | Accuracy= 79.8136645962733
Epoch: 2280 | TrainingLoss= 0.8253925442695618 | TestingLoss= 0.1980602741241455 | Accuracy= 79.8136645962733
Epoch: 2290 | TrainingLoss= 0.8221171498298645 | TestingLoss= 0.1969365030527115 | Accuracy= 79.8136645962733
Epoch: 2300 | TrainingLoss= 0.818870484828949 | TestingLoss= 0.1957547515630722 | Accuracy= 79.8136645962733
Epoch: 2310 | TrainingLoss= 0.8156481981277466 | TestingLoss= 0.19458472728729248 | Accuracy= 79.8136645962733
Epoch: 2320 | TrainingLoss= 0.8124505281448364 | TestingLoss= 0.19353094696998596 | Accuracy= 80.12422360248446
Epoch: 2330 | TrainingLoss= 0.80927973985672 | TestingLoss= 0.19252397119998932 | Accuracy= 80.12422360248446
Epoch:

Epoch: 2990 | TrainingLoss= 0.6392866373062134 | TestingLoss= 0.17303992807865143 | Accuracy= 80.74534161490683
Epoch: 3000 | TrainingLoss= 0.6372297406196594 | TestingLoss= 0.17322391271591187 | Accuracy= 80.74534161490683
Epoch: 3010 | TrainingLoss= 0.6351798176765442 | TestingLoss= 0.17363016307353973 | Accuracy= 80.74534161490683
Epoch: 3020 | TrainingLoss= 0.6331472992897034 | TestingLoss= 0.17386305332183838 | Accuracy= 80.74534161490683
Epoch: 3030 | TrainingLoss= 0.631136953830719 | TestingLoss= 0.1740722954273224 | Accuracy= 80.74534161490683
Epoch: 3040 | TrainingLoss= 0.6291427612304688 | TestingLoss= 0.1741001158952713 | Accuracy= 80.74534161490683
Epoch: 3050 | TrainingLoss= 0.6271565556526184 | TestingLoss= 0.17424540221691132 | Accuracy= 80.74534161490683
Epoch: 3060 | TrainingLoss= 0.6251897811889648 | TestingLoss= 0.17469266057014465 | Accuracy= 80.43478260869566
Epoch: 3070 | TrainingLoss= 0.6232399344444275 | TestingLoss= 0.1748124361038208 | Accuracy= 80.43478260869

...

Epoch: 5950 | TrainingLoss= 0.41063815355300903 | TestingLoss= 0.1712983399629593 | Accuracy= 81.36645962732919
Epoch: 5960 | TrainingLoss= 0.41038158535957336 | TestingLoss= 0.1717556267976761 | Accuracy= 81.36645962732919
Epoch: 5970 | TrainingLoss= 0.41018784046173096 | TestingLoss= 0.17145667970180511 | Accuracy= 81.36645962732919
Epoch: 5980 | TrainingLoss= 0.4099052846431732 | TestingLoss= 0.17204441130161285 | Accuracy= 81.36645962732919
Epoch: 5990 | TrainingLoss= 0.4096568822860718 | TestingLoss= 0.17209087312221527 | Accuracy= 81.36645962732919
Epoch: 6000 | TrainingLoss= 0.4094236195087433 | TestingLoss= 0.1726146787405014 | Accuracy= 81.36645962732919
Epoch: 6010 | TrainingLoss= 0.409211665391922 | TestingLoss= 0.1726788729429245 | Accuracy= 81.36645962732919
Epoch: 6020 | TrainingLoss= 0.4089735150337219 | TestingLoss= 0.1740015745162964 | Accuracy= 81.36645962732919
Epoch: 6030 | TrainingLoss= 0.40870916843414307 | TestingLoss= 0.173996701836586 | Accuracy= 81.36645962732

Epoch: 6690 | TrainingLoss= 0.3950364887714386 | TestingLoss= 0.1825639307498932 | Accuracy= 81.67701863354037
Epoch: 6700 | TrainingLoss= 0.39484620094299316 | TestingLoss= 0.18261614441871643 | Accuracy= 81.67701863354037
Epoch: 6710 | TrainingLoss= 0.39468899369239807 | TestingLoss= 0.18285664916038513 | Accuracy= 81.67701863354037
Epoch: 6720 | TrainingLoss= 0.39448872208595276 | TestingLoss= 0.1824960559606552 | Accuracy= 81.67701863354037
Epoch: 6730 | TrainingLoss= 0.3943028748035431 | TestingLoss= 0.18244780600070953 | Accuracy= 81.67701863354037
Epoch: 6740 | TrainingLoss= 0.394111305475235 | TestingLoss= 0.1819021999835968 | Accuracy= 81.67701863354037
Epoch: 6750 | TrainingLoss= 0.39397481083869934 | TestingLoss= 0.1817091405391693 | Accuracy= 81.67701863354037
Epoch: 6760 | TrainingLoss= 0.3937547504901886 | TestingLoss= 0.18165366351604462 | Accuracy= 81.67701863354037
Epoch: 6770 | TrainingLoss= 0.393561989068985 | TestingLoss= 0.1815943568944931 | Accuracy= 81.6770186335

Epoch: 7430 | TrainingLoss= 0.38322943449020386 | TestingLoss= 0.19056978821754456 | Accuracy= 81.67701863354037
Epoch: 7440 | TrainingLoss= 0.3830864131450653 | TestingLoss= 0.19127994775772095 | Accuracy= 81.67701863354037
Epoch: 7450 | TrainingLoss= 0.382956862449646 | TestingLoss= 0.1914031207561493 | Accuracy= 81.67701863354037
Epoch: 7460 | TrainingLoss= 0.382840633392334 | TestingLoss= 0.19089539349079132 | Accuracy= 81.67701863354037
Epoch: 7470 | TrainingLoss= 0.3826967775821686 | TestingLoss= 0.19083502888679504 | Accuracy= 81.67701863354037
Epoch: 7480 | TrainingLoss= 0.3825625479221344 | TestingLoss= 0.19111301004886627 | Accuracy= 81.67701863354037
Epoch: 7490 | TrainingLoss= 0.3824135959148407 | TestingLoss= 0.191472128033638 | Accuracy= 81.67701863354037
Epoch: 7500 | TrainingLoss= 0.3823256194591522 | TestingLoss= 0.19150258600711823 | Accuracy= 81.67701863354037
Epoch: 7510 | TrainingLoss= 0.3821737766265869 | TestingLoss= 0.1909283995628357 | Accuracy= 81.677018633540

Epoch: 8170 | TrainingLoss= 0.374654620885849 | TestingLoss= 0.19635817408561707 | Accuracy= 81.98757763975155
Epoch: 8180 | TrainingLoss= 0.37453338503837585 | TestingLoss= 0.19593317806720734 | Accuracy= 81.98757763975155
Epoch: 8190 | TrainingLoss= 0.3744199275970459 | TestingLoss= 0.19610922038555145 | Accuracy= 81.98757763975155
Epoch: 8200 | TrainingLoss= 0.37430670857429504 | TestingLoss= 0.1955142319202423 | Accuracy= 81.98757763975155
Epoch: 8210 | TrainingLoss= 0.37420305609703064 | TestingLoss= 0.19602692127227783 | Accuracy= 81.98757763975155
Epoch: 8220 | TrainingLoss= 0.3741321563720703 | TestingLoss= 0.19593635201454163 | Accuracy= 81.98757763975155
Epoch: 8230 | TrainingLoss= 0.3740120530128479 | TestingLoss= 0.19628207385540009 | Accuracy= 81.98757763975155
Epoch: 8240 | TrainingLoss= 0.37390005588531494 | TestingLoss= 0.19639523327350616 | Accuracy= 81.98757763975155
Epoch: 8250 | TrainingLoss= 0.373801052570343 | TestingLoss= 0.19637072086334229 | Accuracy= 81.987577

Epoch: 8910 | TrainingLoss= 0.36773011088371277 | TestingLoss= 0.18890829384326935 | Accuracy= 81.98757763975155
Epoch: 8920 | TrainingLoss= 0.36762112379074097 | TestingLoss= 0.1889440417289734 | Accuracy= 81.98757763975155
Epoch: 8930 | TrainingLoss= 0.36752501130104065 | TestingLoss= 0.18887211382389069 | Accuracy= 81.98757763975155
Epoch: 8940 | TrainingLoss= 0.3674541115760803 | TestingLoss= 0.18911121785640717 | Accuracy= 81.98757763975155
Epoch: 8950 | TrainingLoss= 0.3673676550388336 | TestingLoss= 0.18814043700695038 | Accuracy= 81.98757763975155
Epoch: 8960 | TrainingLoss= 0.36730602383613586 | TestingLoss= 0.18889220058918 | Accuracy= 81.98757763975155
Epoch: 8970 | TrainingLoss= 0.36723315715789795 | TestingLoss= 0.18874667584896088 | Accuracy= 81.98757763975155
Epoch: 8980 | TrainingLoss= 0.36713147163391113 | TestingLoss= 0.18829654157161713 | Accuracy= 81.98757763975155
Epoch: 8990 | TrainingLoss= 0.36703258752822876 | TestingLoss= 0.18871980905532837 | Accuracy= 81.9875

In [ ]:
from pathlib import Path

    # 1. Create models directory 
    MODEL_PATH = Path(f"data/classLen- {len(ClassList)}/embeding-{n}")
    MODEL_PATH.mkdir(parents=True, exist_ok=True)

    # 2. Create model save path 
    MODEL_NAME = f"predictFacesModelAcc{testAcc:.2f}_With{n}Embeds.pth"
    MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

    # 3. Save the model state dict 
    print(f"Saving model to: {MODEL_SAVE_PATH}")
    torch.save(obj=model.state_dict(), # only saving the state_dict() only saves the models learned parameters
               f=MODEL_SAVE_PATH)